<a href="https://colab.research.google.com/github/grewe/covidID_SocDist/blob/master/inferenceD0DPDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
models_dir ="/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/models"

In [4]:
!printenv
%pwd
%cd {models_dir}
%pwd


CUDNN_VERSION=7.6.5.32
LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64
CLOUDSDK_PYTHON=python3
_=/usr/bin/printenv
LANG=en_US.UTF-8
HOSTNAME=509fd769ba8d
OLDPWD=/
CLOUDSDK_CONFIG=/content/.config
NVIDIA_VISIBLE_DEVICES=all
DATALAB_SETTINGS_OVERRIDES={"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\"172.28.0.2\""]}
ENV=/root/.bashrc
PAGER=cat
NCCL_VERSION=2.7.8
TF_FORCE_GPU_ALLOW_GROWTH=true
JPY_PARENT_PID=25
NO_GCE_CHECK=True
PWD=/content
HOME=/root
LAST_FORCED_REBUILD=20200910
CLICOLOR=1
DEBIAN_FRONTEND=noninteractive
LIBRARY_PATH=/usr/local/cuda/lib64/stubs
GCE_METADATA_TIMEOUT=0
GLIBCPP_FORCE_NEW=1
TBE_CREDS_ADDR=172.28.0.1:8008
TERM=xterm-color
SHELL=/bin/bash
GCS_READ_CACHE_BLOCK_SIZE_MB=16
PYTHONWARNINGS=ignore:::pip._internal.cli.base_command
MPLBACKEND=module://ipykernel.pylab.backend_inline
CUDA_PKG_VERSION=10-1=10.1.243-1
CUDA_VERSION=10.1.243
NVIDIA_DRIVER_CAPABILITIES=compute,utility
SHLVL=2
PYTHONPATH=/env/python
NVID

'/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/models'

In [6]:
model_dir = "/content/drive/'My Drive'/SocialDistance/DPDM/EfficientDetD0/training/"
models_research_dir = "/content/drive/'My Drive'/SocialDistance/DPDM/EfficientDetD0/models/research/"

In [7]:
# MUST do each time restart colab kernel
# INSTALL Object Detection API inside the Colab, as sits above TF2
# this will take several minutes as it will copy over all of the object_detection files
# you have mounted in project_folder/models/research/object_detection
# as you can see by the output of this cell it copies them into the colab
# environment at build/lib/object_detection

#this this is what I need to make the object_detection get installed

print(models_research_dir)
%cd {models_research_dir}
#%ls
#!python setup.py install   # from the models/reasearch


#install tf_slim
!pip install tf_slim

/content/drive/'My Drive'/SocialDistance/DPDM/EfficientDetD0/models/research/
/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/models/research
     |████████████████████████████████| 358kB 2.4MB/s 


In [10]:
!pwd
%cd {models_dir}
%pwd

/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/models/research
/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/models


'/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/models'

In [11]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import os
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
import cv2

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [12]:
def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def load_grayscale_image_into_numpy_array(path):
  # read in image as grayscale cv2 image
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  # convert image to rgb
  backtorgb = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
  return backtorgb.astype(np.uint8)

In [13]:
def load_rgb_image_into_numpy_array(path):
  # read in image as grayscale cv2 image
  img = cv2.imread(path, cv2.IMREAD_COLOR)
  return img
  # convert image to rgb
  #backtorgb = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
  #return backtorgb.astype(np.uint8)

In [14]:
pipeline_file = '/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/fine_tuned_model/pipeline.config'
#recover our saved model
pipeline_config = pipeline_file
#generally you want to put the last ckpt from training in here
model_dir = '/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/training/train/ckpt-41'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
#from object_detection.builders import model_builder
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/training/ckpt-41'))

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [16]:
import cv2
import glob
import os
from PIL import Image

# wherever your test images now live 
TEST_IMAGE_PATHS = glob.glob('/content/drive/My Drive/SocialDistance/DPDM/EfficientDetD0/Data/output/*.jpg')

for image_path in TEST_IMAGE_PATHS:
    # load image into np array
    image_np = load_rgb_image_into_numpy_array(image_path)
    # convert to input tensor ?
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    # run detect_fun on input tensor
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    # start labels at 1 instead of 0?
    label_id_offset = 1
    # make a copy of np image
    image_np_with_detections = image_np.copy()
    # add bounding boxes and labels to image
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.5,
          agnostic_mode=False,
    )
    # get the output path for writing results (i'm tacking on bb_ to the names so i can easily identify them as
    # having been annotated)
    output_path = os.path.join(os.path.dirname(image_path), "results", "bb_" + os.path.basename(image_path))
    # convert to a PIL Image for writing
    save_image = Image.fromarray(image_np_with_detections)
    # write to file
    save_image.save(output_path)